In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

import acquire
import prepare
import env

from wrangle_zillow import wrangle_zillow_data

## acquire

In [2]:
query ='''
select 
    prop.parcelid
    , pred.logerror
    , pred.transactiondate
    , bathroomcnt
    , bedroomcnt
    , calculatedfinishedsquarefeet
    , fips
    , latitude
    , longitude
    , lotsizesquarefeet
    , regionidcity
    , regionidcounty
    , regionidneighborhood
    , regionidzip
    , yearbuilt
    , structuretaxvaluedollarcnt
    , taxvaluedollarcnt
    , landtaxvaluedollarcnt
    , taxamount
from properties_2017 prop
inner join predictions_2017 pred on prop.parcelid = pred.parcelid
where propertylandusetypeid = 261;
'''

df = pd.read_sql(query, env.get_url('zillow'))

In [3]:
#df = pd.read_csv('zillow_data.csv')

In [4]:
#df = df.drop(columns=['Unnamed: 0'])

### Goal: Improve our original estimate of the log error by using clustering methodologies.

## Acquisition, Prep, and Initial Exploration
Using the notebook and files you created during the exercises make any changes, additions, etc. you want at this point. NOTE: You will NOT be splitting into train and test at this point.

Ideas:

   1. Data types:

        - Write a function that takes in a dataframe and a list of column names and returns the dataframe with the datatypes of those columns changed to a non-numeric type.

In [5]:
df.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidcounty,regionidneighborhood,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
0,14297519,0.025595,2017-01-01,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,53571.0,1286.0,NaN,96978.0,1998.0,485713.0,1023282.0,537569.0,11013.72
1,17052889,0.055619,2017-01-01,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,13091.0,2061.0,NaN,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48
2,14186244,0.005383,2017-01-01,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,21412.0,1286.0,NaN,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30
3,12177905,-0.103410,2017-01-01,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,396551.0,3101.0,NaN,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51
4,12095076,-0.001011,2017-01-01,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,47019.0,3101.0,274684.0,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26


In [6]:
# it works!
cols = ['bathroomcnt', 'latitude']
def change_data(df, cols):
    """
    takes a dataframe and a list of columns and it 
    converts the columns listed that are in the dataframe into 
    objects in the same dataframe
    """
    newdf = pd.DataFrame(df, columns=cols)
    df = df.drop(columns=cols)
    newdf = newdf.astype(object)
    df = pd.concat([df, newdf], axis=1)
    return df
    
        



#zillow = df.copy()
#zillow = zillow.astype(object)

   - Use this function to appropriately transform any numeric columns that should not be treated as numbers.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52442 entries, 0 to 52441
Data columns (total 19 columns):
parcelid                        52442 non-null int64
logerror                        52442 non-null float64
transactiondate                 52442 non-null object
bathroomcnt                     52442 non-null float64
bedroomcnt                      52442 non-null float64
calculatedfinishedsquarefeet    52360 non-null float64
fips                            52442 non-null float64
latitude                        52442 non-null float64
longitude                       52442 non-null float64
lotsizesquarefeet               52073 non-null float64
regionidcity                    51405 non-null float64
regionidcounty                  52442 non-null float64
regionidneighborhood            19033 non-null float64
regionidzip                     52416 non-null float64
yearbuilt                       52326 non-null float64
structuretaxvaluedollarcnt      52358 non-null float64
taxvaluedollar

>upon more inspection it looks like we could use this function to convert zipcodes parcelid id fips, rawcensustractbloc, regionid's,

In [8]:
df.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,regionidcity,regionidcounty,regionidneighborhood,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
0,14297519,0.025595,2017-01-01,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,53571.0,1286.0,NaN,96978.0,1998.0,485713.0,1023282.0,537569.0,11013.72
1,17052889,0.055619,2017-01-01,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,13091.0,2061.0,NaN,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48
2,14186244,0.005383,2017-01-01,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,21412.0,1286.0,NaN,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30
3,12177905,-0.103410,2017-01-01,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,396551.0,3101.0,NaN,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51
4,12095076,-0.001011,2017-01-01,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,47019.0,3101.0,274684.0,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26


In [9]:
cols = ['parcelid','fips', 'regionidcity', 'regionidcounty', 'regionidzip',
       'regionidneighborhood',]
zillow = change_data(df, cols)

In [10]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52442 entries, 0 to 52441
Data columns (total 19 columns):
logerror                        52442 non-null float64
transactiondate                 52442 non-null object
bathroomcnt                     52442 non-null float64
bedroomcnt                      52442 non-null float64
calculatedfinishedsquarefeet    52360 non-null float64
latitude                        52442 non-null float64
longitude                       52442 non-null float64
lotsizesquarefeet               52073 non-null float64
yearbuilt                       52326 non-null float64
structuretaxvaluedollarcnt      52358 non-null float64
taxvaluedollarcnt               52441 non-null float64
landtaxvaluedollarcnt           52441 non-null float64
taxamount                       52438 non-null float64
parcelid                        52442 non-null object
fips                            52442 non-null object
regionidcity                    51405 non-null object
regionidcounty 

   2. Missing Values: Impute the values in land square feet.

In [11]:
zillow.sample()

,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,latitude,longitude,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,parcelid,fips,regionidcity,regionidcounty,regionidzip,regionidneighborhood
36190,0.12837,2017-06-30,3.0,2.0,2208.0,34099798.0,-118277433.0,5568.0,1962.0,136768.0,611479.0,474711.0,7374.74,11945676,6037,12447,3101,96008,274359


In [12]:
round(zillow.isna().sum()/len(zillow)*100,2)

logerror                         0.00
transactiondate                  0.00
bathroomcnt                      0.00
bedroomcnt                       0.00
calculatedfinishedsquarefeet     0.16
latitude                         0.00
longitude                        0.00
lotsizesquarefeet                0.70
yearbuilt                        0.22
structuretaxvaluedollarcnt       0.16
taxvaluedollarcnt                0.00
landtaxvaluedollarcnt            0.00
taxamount                        0.01
parcelid                         0.00
fips                             0.00
regionidcity                     1.98
regionidcounty                   0.00
regionidzip                      0.05
regionidneighborhood            63.71
dtype: float64

  - For land square feet, the goal is to impute the missing values by creating a linear model where landtaxvaluedollarcnt is the x-variable and the output/y-variable is the estimated land square feet.

In [13]:
preml = zillow[['lotsizesquarefeet', 'landtaxvaluedollarcnt']].fillna(0)

In [14]:
lm = LinearRegression()
X = preml[['landtaxvaluedollarcnt']]
y = preml[['lotsizesquarefeet']]
lm.fit(X, y)
pred = lm.predict(X)
scores = []

scores.append(r2_score(X, y))

   - We'll then use this model to make predictions and fill in the missing values.

In [15]:
zillow['predict_lotsize'] = lm.predict(X).astype(int)

In [16]:
zillow.head()

,logerror,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,latitude,longitude,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,parcelid,fips,regionidcity,regionidcounty,regionidzip,regionidneighborhood,predict_lotsize
0,0.025595,2017-01-01,3.5,4.0,3100.0,33634931.0,-117869207.0,4506.0,1998.0,485713.0,1023282.0,537569.0,11013.72,14297519,6059,53571,1286,96978,NaN,11828
1,0.055619,2017-01-01,1.0,2.0,1465.0,34449266.0,-119281531.0,12647.0,1967.0,88000.0,464000.0,376000.0,5672.48,17052889,6111,13091,2061,97099,NaN,11378
2,0.005383,2017-01-01,2.0,3.0,1243.0,33886168.0,-117823170.0,8432.0,1962.0,85289.0,564778.0,479489.0,6488.30,14186244,6059,21412,1286,97078,NaN,11666
3,-0.103410,2017-01-01,3.0,4.0,2376.0,34245180.0,-118240722.0,13038.0,1970.0,108918.0,145143.0,36225.0,1777.51,12177905,6037,396551,3101,96330,NaN,10431
4,-0.001011,2017-01-01,3.0,4.0,2962.0,34145202.0,-118179824.0,63000.0,1950.0,276684.0,773303.0,496619.0,9516.26,12095076,6037,47019,3101,96293,274684,11714


   - Write a function that accepts the zillow data frame and returns the data frame with the missing values filled in.

In [17]:
# commented function out to preserve computation power
#zillow['lotsizesquarefeet'] = zillow.apply(lambda row: row['predict_lotsize'] if np.isnan(row['lotsizesquarefeet']) else zillow['lotsizesquarefeet'], axis=1)

   3. Missing Values: Of the remaining missing values, can they be imputed or otherwise estimated?

        - Impute those that can be imputed with the method you feel best fits the attribute.
        - Decide whether to remove the rows or columns of any that cannot be reasonably imputed.
        - Document your reasons for the decisions on how to handle each of those.

In [18]:
round(zillow.isna().sum()/len(zillow)*100,2)

logerror                         0.00
transactiondate                  0.00
bathroomcnt                      0.00
bedroomcnt                       0.00
calculatedfinishedsquarefeet     0.16
latitude                         0.00
longitude                        0.00
lotsizesquarefeet                0.00
yearbuilt                        0.22
structuretaxvaluedollarcnt       0.16
taxvaluedollarcnt                0.00
landtaxvaluedollarcnt            0.00
taxamount                        0.01
parcelid                         0.00
fips                             0.00
regionidcity                     1.98
regionidcounty                   0.00
regionidzip                      0.05
regionidneighborhood            63.71
predict_lotsize                  0.00
dtype: float64

In [19]:
# we're going to drop regionidneighborhood because it's missing over 50% of it's data

In [20]:
zillow = zillow.drop(columns=['regionidneighborhood'])

In [21]:
# we're going to check out regionidcity and see if that column is vital to our analysis

In [22]:
zillow.regionidcity.value_counts()

12447.0    11452
5534.0      1795
40227.0     1492
46298.0     1428
16764.0     1087
           ...  
32927.0        3
31134.0        2
21395.0        1
36078.0        1
10815.0        1
Name: regionidcity, Length: 175, dtype: int64

In [23]:
# I'm not sure what these codes are, so we're going to drop this 
#columns for now and if we need it later i'll impute
zillow = zillow.drop(columns=['regionidcity'])

In [24]:
# We're going to use the median to fill taxamount
zillow['taxamount'] = zillow['taxamount'].fillna(zillow.taxamount.median())

In [25]:
# fillna's with median for yearbuilt
zillow['yearbuilt'] = zillow['yearbuilt'].fillna(zillow.yearbuilt.median())


In [26]:
# fillna's with median for structuretaxvaluedollarcnt
zillow['structuretaxvaluedollarcnt'] = zillow['structuretaxvaluedollarcnt'].fillna(zillow.structuretaxvaluedollarcnt.median())

In [27]:
# fillna's with median for calculatedfinishedsquarefeet
zillow['calculatedfinishedsquarefeet'] = zillow['calculatedfinishedsquarefeet'].fillna(zillow.calculatedfinishedsquarefeet.median())

In [29]:
zillow['regionidzip'] = zillow['regionidzip'].fillna(zillow.regionidzip.median())

In [30]:
round(zillow.isna().sum()/len(zillow)*100,2)

logerror                        0.0
transactiondate                 0.0
bathroomcnt                     0.0
bedroomcnt                      0.0
calculatedfinishedsquarefeet    0.0
latitude                        0.0
longitude                       0.0
lotsizesquarefeet               0.0
yearbuilt                       0.0
structuretaxvaluedollarcnt      0.0
taxvaluedollarcnt               0.0
landtaxvaluedollarcnt           0.0
taxamount                       0.0
parcelid                        0.0
fips                            0.0
regionidcounty                  0.0
regionidzip                     0.0
predict_lotsize                 0.0
dtype: float64

In [31]:
# turning our df into a csv a
#zillow.to_csv('zillow.csv')

In [32]:
zillow = pd.read_csv('zillow.csv')

In [35]:
zillow = zillow.drop(columns=['Unnamed: 0'])

6037.0    33911
6059.0    14136
6111.0     4395
Name: fips, dtype: int64

   4. Outliers: Original from exercises. Adapt as you see fit.

        - Write a function that accepts a series (i.e. one column from a data frame) and summarizes how many outliers are in the series. This function should accept a second parameter that determines how outliers are detected, with the ability to detect outliers in 3 ways: IQR, standard deviations (z-score), percentiles)

   5. Use your function defined above to identify columns where you should handle the outliers.

   6. Write a function that accepts the zillow data frame and removes the outliers. You should make a decision and document how you will remove outliers.

   7. Is there erroneous data you have found that you need to remove or repair? If so, take action.

   8. Are there outliers you want to "squeeze in" to a max value? (e.g. all bathrooms > 6 => bathrooms = 6). If so, make those changes.

# Exploration with Clustering
## Cluster the Target Variable
    Why? By reducing the noise of the continuous variable, we can possibly see trends easier by turning this continuous variable into clusters and then comparing those clusters with respect to other variables through visualizations or tests.

    Perform clustering with logerror as the only feature used in the clustering algorithm. Decide on a number of clusters to use, and store the cluster predictions back onto your data frame as cluster_target. Look at the centroids that were produced in this process. What do they tell you?

    Use the produced clusters to help you explore through visualization how logerror relates to other variables. (A common way to do this is to use color to indicate the cluster id, and the other variables can be your x-axis and y-axis. (hint: look at your swarmplot function)).

## Cluster Independent Variables
   You should also perform some clustering based on a number of independent variables. Create and evaluate several clustering models based on subsets of the independent variables. Here are some ideas:

   - Location, that is, latitude and longitude
   - Size (finished square feet)
   - Location and size
   - Be sure to use these new clusters in exploring your data, and interpret what these clusters tell you.

## Test the Significance of Clusters
    Use statistical testing methods to determine whether the clusters you have created are significant in terms of their relationship to logerror.

# Modeling
## Feature Engineering
   1. Remove variables that are not needed, wanted, useful, or are redundant.
   2. Add any features you think may be useful.
   3. Split your data into training and test sets.
   4. Create subsets of data if you would like to create multiple models and then merge (such as, a different model for each cluster or for each county).

# Model Selection
   1. Train at least 3 different models (a model is different if there are changes in one or more of the following: features, hyper-parameters, algorithm). Create object, fit, predict & evaluate. Use mean absolute error or mean squared error to evaluate. Also, try regression algorithms you have not used before.
   2. Evaluate your best model on your test data set to get an idea of your model's out of sample error.